This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [9]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
import apikeys
API_KEY = apikeys.quandl

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [10]:
# First, import the relevant modules
import requests
import json
import logging



In [22]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
logging.basicConfig(level=logging.DEBUG, filename='miniProj.log', filemode='w', format='%(name)s - %(levelname)s - %(message)s')

url_u = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-01-02&api_key="+API_KEY
r = requests.get(url_u)
print(r.json())



{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2017-01-01', 'end_date': '2017-01-02', 'frequency': 'daily', 'data': [['2017-01-02', 34.99, 35.94, 34.99, 35.8, None, 44700.0, 1590561.0, None, None, None]], 'collapse': None, 'order': None}}


In [5]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
logging.debug(r.json)

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [21]:
url_u = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key="+API_KEY
r = requests.get(url_u)

col_names = r.json()['dataset_data']['column_names']
logging.debug(col_names)

dict_test=dict(r.json()['dataset_data'])

type(dict_test['data'])

#initialization
minOpen = float(dict_test['data'][0][1])
maxOpen = float(dict_test['data'][0][1])
maxdayChange = 0
daychange = 0
closeChange = dict_test['data'][0][4] - dict_test['data'][1][4]
maxcloseChange = closeChange
lastClose = dict_test['data'][0][4]
date = dict_test['data'][0][0]
tradedVol = 0
count = 0
tradedVolList = []  #empty list


#loop to iterate through each date data
for x in dict_test['data']:
    logging.debug(x)
    #Minimum Open and Max Open
    if type(x[1]) is float:
        if(x[1] < float(minOpen)):
            minOpen = x[1]
        elif(x[1] > maxOpen):
            maxOpen = x[1]
    #Day Change Calculations
    if (type(x[2]) is float) and (type(x[3]) is float) :
        daychange = x[2] - x[3]
        logging.debug("change:"+str(daychange))
    #Max Change in a Day
    if(daychange > maxdayChange):
        maxdayChange = daychange
    #Change between Close vals in two days
    if(type(x[4]) is float):
        closeChange = lastClose - x[4]
        lastClose = x[4]
    #Max Close change in two days
    if(closeChange > maxcloseChange):
        date = x[0]
        maxcloseChange = closeChange
    #TradedVol avg and list creation for median calcs
    if (type(x[6]) is float) :
        tradedVol = tradedVol + x[6]
        tradedVolList = tradedVolList + [tradedVol]
    count = count + 1

    
#calculate median
def median(listofelems):
    sort = sorted(listofelems)
    num = len(listofelems)
    return (sum(sort[int(num/2)-1:int(num/2)+1])/2.0, sort[int(num/2)])[num % 2] if num else None
    
    
print("minOpen:"+str(minOpen))
print("maxOpen:"+str(maxOpen))
print("maxdayChange:"+str(maxdayChange))
print("maxcloseChange:"+str(maxcloseChange))
print("dateMaxCloseChange:"+ str(date))
print("avgTradVol:" + str(tradedVol/count))
print("totalDays:" + str(count))
print("Median tradedVol:" + str(median(tradedVolList)))

minOpen:34.0
maxOpen:53.11
maxdayChange:2.8100000000000023
maxcloseChange:1.7199999999999989
dateMaxCloseChange:2017-05-10
avgTradVol:89124.33725490196
totalDays:255
Median tradedVol:11049198.0
